In [4]:
import sys
!{sys.executable} -m pip install pymongo
!{sys.executable} -m pip install pyyaml

In [5]:
# Get Mongo database
from yaml import load
from pymongo import MongoClient
from getsecret import getsecret

client = MongoClient(getsecret("MONGODB_URI"))
db = client[getsecret("DB_NAME")]

In [8]:
# Look through errors.
for name in db.list_collection_names():
    if "error" in name:
        for error in db[name].find({"timestamp":{"$gt": 1534279887226}}):
            print(name)
            print(error)

881a0d247c847afb93b228ca_error
{'_id': ObjectId('5b73531739063900145c6bd7'), 'error': '{"userID":"881a0d247c847afb93b228ca","name":"Error","nativeException":{},"message":"The application crashed because of an uncaught exception. You can look at \\"stackTrace\\" or \\"nativeException\\" for more detailed information about the exception.","stackTrace":"An uncaught Exception occurred on \\"main\\" thread.\\njava.lang.RuntimeException: Error receiving broadcast Intent { act=android.intent.action.USER_PRESENT flg=0x24200010 } in com.tns.gen.android.content.BroadcastReceiver_AccessibilityService_63_56_@a91c206\\n\\tat android.app.LoadedApk$ReceiverDispatcher$Args.lambda$-android_app_LoadedApk$ReceiverDispatcher$Args_52851(LoadedApk.java:1332)\\n\\tat android.app.-$Lambda$aS31cHIhRx41653CMnd4gZqshIQ.$m$7(Unknown Source:4)\\n\\tat android.app.-$Lambda$aS31cHIhRx41653CMnd4gZqshIQ.run(Unknown Source:39)\\n\\tat android.os.Handler.handleCallback(Handler.java:790)\\n\\tat android.os.Handler.dispat